### Test camera availability

In [4]:
import cv2

def test_cameras():
    for i in range(5):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            print(f"Camera {i} is available")
            ret, frame = cap.read()
            if ret:
                print(f"  Resolution: {frame.shape[1]}x{frame.shape[0]}")
                cv2.imshow(f"Camera {i}", frame)
                cv2.waitKey(1000)
                cv2.destroyAllWindows()
            cap.release()
        else:
            print(f"Camera {i} NOT available")

test_cameras()

Camera 0 is available
  Resolution: 640x480
Camera 1 NOT available
Camera 2 NOT available
Camera 3 NOT available
Camera 4 NOT available


### Load and Test the Model with Webcam

In [2]:
import cv2
import numpy as np
import tensorflow as tf

# Load trained model
model = tf.keras.models.load_model("./NN model.keras")

# Start webcam
cap = cv2.VideoCapture(0)

prediction_text = "Show a digit & press SPACE"
print("Press SPACE to predict, Q to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    # ROI for digit
    x1, y1, x2, y2 = 100, 50, 500, 450
    roi = frame[y1:y2, x1:x2]
    cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)

    # Preprocessing|
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(
        blur, 0, 255,
        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    digit_img = None
    if contours:
        # Largest contour = digit
        largest = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest)

        if area > 1500:     # better threshold
            x, y, w, h = cv2.boundingRect(largest)
            digit = thresh[y:y+h, x:x+w]

            # Resize to MNIST size
            digit = cv2.resize(digit, (28, 28))

            # Clean up: normalize
            digit_img = digit.astype("float32") / 255.0

            cv2.rectangle(roi, (x,y), (x+w,y+h), (255,0,0), 2)

    key = cv2.waitKey(1) & 0xFF

    # SPACE → RUN prediction
    if key == 32 and digit_img is not None:
        img = digit_img.reshape(1, 784)

        pred = model.predict(img, verbose=0)
        prediction = np.argmax(pred)
        confidence = pred[0][prediction]
        print(pred)
        print(prediction)
        print(confidence)

        if confidence >= 0.95:
            prediction_text = f"Pred: {prediction}  ({confidence:.2f})"
        else:
            prediction_text = "Could not catch"

    # Quit
    if key == ord('q'):
        break

    # Draw prediction on screen
    cv2.putText(
        frame,
        prediction_text,
        (150, 80),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.2,
        (0,0,255),
        2
    )

    # Show windows
    cv2.imshow("MNIST Digit Recognition", frame)
    cv2.imshow("Threshold", thresh)

cap.release()
cv2.destroyAllWindows()


Press SPACE to predict, Q to quit
[[2.77489498e-19 1.54013672e-12 9.75476027e-01 7.81753968e-08
  8.31669706e-27 2.45239064e-02 4.22511491e-18 5.63863626e-13
  1.23468427e-08 1.35198715e-14]]
2
0.975476
[[7.7081202e-23 3.2135478e-11 9.9999988e-01 1.5037142e-07 6.8597833e-35
  1.4640774e-08 1.3585819e-24 2.0242644e-14 2.0061080e-13 4.2099858e-20]]
2
0.9999999
[[2.2019289e-22 3.0014758e-11 1.0000000e+00 4.0939208e-09 4.7696536e-33
  1.7493193e-08 1.8667923e-23 1.1836479e-13 9.9901023e-13 3.2567163e-19]]
2
1.0
[[3.0319395e-18 2.2006823e-11 6.5227103e-01 1.6745274e-04 3.6699084e-32
  3.4756151e-01 3.6555990e-18 1.8383350e-12 5.2501353e-10 1.5001657e-15]]
2
0.65227103
[[2.9255056e-18 2.8015749e-11 9.9170107e-01 4.9179430e-06 3.8615173e-31
  8.2940599e-03 6.5621903e-19 1.5068230e-11 3.7008396e-10 8.2745071e-16]]
2
0.99170107
[[2.7292804e-19 2.4053708e-09 9.9993908e-01 2.4289702e-06 5.8694274e-29
  5.8432990e-05 2.2747792e-18 7.0988121e-10 1.6593818e-11 2.7679083e-14]]
2
0.9999391
[[1.5745055